In [1]:
import os
from _src import LOG_DIR, DB_DIR
from _src import search_log_files

In [6]:
import os
import optuna
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go

# Function to convert hex to rgba
def hex_to_rgba(hex_color, alpha=0.2):
    hex_color = hex_color.lstrip('#')
    rgb = tuple(int(hex_color[i:i+2], 16) for i in (0, 2, 4))
    return f'rgba({rgb[0]}, {rgb[1]}, {rgb[2]}, {alpha})'

# Plot and save figure function
def plot_and_save_fig(sampler_files, db_folder, color_scale, save_path, width=800, height=600, title=""):
    fig = go.Figure()

    # Loop over each sampler and assign colors from the Plotly qualitative scale
    for idx, (sampler, db_files) in enumerate(sampler_files.items()):
        dataframes = []
        for db_file in db_files:
            path = os.path.join(db_folder, db_file)
            storage_url = f"sqlite:///{path}"

            study_name = db_file.split('_bo_')[1].replace('.db', '')
            study_name = "bo_" + study_name if "parafac" in sampler else "sphere_bo_" + study_name

            study = optuna.load_study(study_name=study_name, storage=storage_url)

            df = study.trials_dataframe().copy()
            if study.direction == optuna.study.StudyDirection.MAXIMIZE:
                df['best_value'] = df['value'].cummax()
            else:
                df['best_value'] = df['value'].cummin()
            dataframes.append(df[['number', 'best_value']])

        merged_df = pd.concat(dataframes, axis=1, keys=[f'seed_{i}' for i in range(len(db_files))])
        best_values = merged_df.xs('best_value', axis=1, level=1)
        mean_best = best_values.mean(axis=1)
        std_best = best_values.std(axis=1)

        upper_bound = mean_best + std_best
        lower_bound = mean_best - std_best

        fig.add_trace(go.Scatter(
            x=best_values.index,
            y=mean_best,
            mode='lines',
            name=f'Mean Best Value ({sampler})',
            line=dict(color=color_scale[idx])
        ))

        fig.add_trace(go.Scatter(
            x=best_values.index,
            y=upper_bound,
            mode='lines',
            line=dict(width=0),
            showlegend=False
        ))

        fig.add_trace(go.Scatter(
            x=best_values.index,
            y=lower_bound,
            mode='lines',
            line=dict(width=0),
            fill='tonexty',
            fillcolor=hex_to_rgba(color_scale[idx]),
            showlegend=False
        ))

    fig.update_layout(
        title=title,
        xaxis_title="Iteration",
        yaxis_title="Best Value",
        template="plotly_white",
        legend=dict(
            orientation="h",
            x=0,
            y=1
        ),
        width=width,
        height=height
    )

    fig.write_image(save_path)


if __name__ == '__main__':
    result_path = "/Users/keisukeonoue/ws/constrained_BO_v2/results_sphere_1"
    db_folder = os.path.join(result_path, "dbs")

    LOG_DIR = "/Users/keisukeonoue/ws/constrained_BO_v2/results_sphere_1/logs"
    dir_path = "/Users/keisukeonoue/ws/constrained_BO_v2/results_sphere_1/dbs"


    problem_setting = {
        "dim": ["dim3", "dim5", "dim7"],
        "model": ["random", "parafac"]
    }

    parafac_setting = {
        "rank": ["rank2"],
        "mask": ["mask0.3"],
        "tradeoff": ["tradeoff3.0"]
    }

    log_files = {dim: {} for dim in problem_setting["dim"]}



    for dim in problem_setting["dim"]:
        for rank in parafac_setting["rank"]:
            for tradeoff in parafac_setting["tradeoff"]:
                sampler_files = {sampler: [] for sampler in problem_setting["model"] if sampler != "parafac"}

                for model in problem_setting["model"]:
                    if model == "parafac":
                        for mask in parafac_setting["mask"]:
                            parafac_name = f"{model}_rank{rank}_mask{mask}_tradeoff{tradeoff}"
                            sampler_files[parafac_name] = []

                            files = search_log_files(dir_path, [dim, rank, model, mask, tradeoff])
                            if len(files) == 5:
                                sampler_files[parafac_name].extend(files)
                    else:
                        files = search_log_files(dir_path, [dim, model])
                        if len(files) == 5:
                            sampler_files[model].extend(files)

                log_files[dim] = sampler_files

                color_scale = px.colors.qualitative.Plotly + px.colors.qualitative.D3 + px.colors.qualitative.G10 + px.colors.qualitative.T10
                save_path = os.path.join(result_path, "images", f"best_values_plot_{dim}_{rank}_{tradeoff}.png")
                
                plot_and_save_fig(
                    sampler_files, 
                    db_folder, 
                    color_scale, 
                    save_path,
                    width=600,
                    height=400,
                    title=f'Sphere of dimension {dim}'
                )


In [3]:
log_files

{'dim3': {'random': ['2024-10-27_09-33-51_sphere_bo_benchmark_random_dim3_seed0.db',
   '2024-10-27_09-34-33_sphere_bo_benchmark_random_dim3_seed1.db',
   '2024-10-27_09-35-17_sphere_bo_benchmark_random_dim3_seed2.db',
   '2024-10-27_09-36-04_sphere_bo_benchmark_random_dim3_seed3.db',
   '2024-10-27_09-36-52_sphere_bo_benchmark_random_dim3_seed4.db'],
  'parafac_rankrank2_maskmask0.3_tradeofftradeoff3.0': ['2024-10-27_00-34-36_bo_parafac_dim3_rank2_mask0.33_tradeoff3.0_seed0.db',
   '2024-10-27_00-34-36_bo_parafac_dim3_rank2_mask0.33_tradeoff3.0_seed1.db',
   '2024-10-27_00-34-36_bo_parafac_dim3_rank2_mask0.33_tradeoff3.0_seed2.db',
   '2024-10-27_00-34-36_bo_parafac_dim3_rank2_mask0.33_tradeoff3.0_seed3.db',
   '2024-10-27_00-34-36_bo_parafac_dim3_rank2_mask0.33_tradeoff3.0_seed4.db']},
 'dim5': {},
 'dim7': {}}

In [5]:
import os
import optuna
from optuna.visualization import plot_optimization_history

# プロットを保存する関数
def save_history_plot(study, save_path):
    fig = plot_optimization_history(study)
    fig.update_layout(title=f"Optimization History: {study.study_name}")
    fig.write_image(save_path)
    print(f"Saved plot to {save_path}")

# データベースファイルのパス
result_path = "/Users/keisukeonoue/ws/constrained_BO_v2/results_sphere_1"
db_folder = os.path.join(result_path, "dbs")
save_folder = os.path.join(result_path, "images")

# 保存先フォルダの作成
os.makedirs(save_folder, exist_ok=True)

# データベースファイルごとにプロットを生成して保存
db_files = [f for f in os.listdir(db_folder) if f.endswith(".db")]

for db_file in db_files:
    db_path = os.path.join(db_folder, db_file)
    storage_url = f"sqlite:///{db_path}"
    
    # ファイル名から study_name を推測
    study_name = db_file.split('_bo_')[1].replace('.db', '')
    study_name = "bo_" + study_name if "parafac" in db_file else "sphere_bo_" + study_name
    
    # Studyを読み込む
    study = optuna.load_study(study_name=study_name, storage=storage_url)
    
    # プロットを保存
    save_path = os.path.join(save_folder, f"{study_name}_history_plot.png")
    save_history_plot(study, save_path)

Saved plot to /Users/keisukeonoue/ws/constrained_BO_v2/results_sphere_1/images/bo_parafac_dim3_rank2_mask0.33_tradeoff3.0_seed2_history_plot.png
Saved plot to /Users/keisukeonoue/ws/constrained_BO_v2/results_sphere_1/images/sphere_bo_benchmark_random_dim3_seed4_history_plot.png
Saved plot to /Users/keisukeonoue/ws/constrained_BO_v2/results_sphere_1/images/bo_parafac_dim7_rank2_mask0.33_tradeoff3.0_seed1_history_plot.png
Saved plot to /Users/keisukeonoue/ws/constrained_BO_v2/results_sphere_1/images/sphere_bo_benchmark_random_dim2_seed2_history_plot.png
Saved plot to /Users/keisukeonoue/ws/constrained_BO_v2/results_sphere_1/images/sphere_bo_benchmark_random_dim2_seed4_history_plot.png
Saved plot to /Users/keisukeonoue/ws/constrained_BO_v2/results_sphere_1/images/sphere_bo_benchmark_random_dim5_seed0_history_plot.png
Saved plot to /Users/keisukeonoue/ws/constrained_BO_v2/results_sphere_1/images/bo_parafac_dim7_rank2_mask0.33_tradeoff3.0_seed4_history_plot.png
Saved plot to /Users/keisukeo